In [14]:
import numpy as np
from matplotlib import pyplot
import pdb
import imp
from pathlib import Path
import rasterio
from rasterio.windows import Window
import torch
from torch import nn
import fastai
from fastai.basic_data import DataBunch
from fastai.basic_train import Learner
from fastai import metrics
from fastai import train
from lib import tools
from multispectral import coords
from multispectral import windows
from multispectral import corine

In [31]:
#imp.reload(coords)
imp.reload(corine)

<module 'multispectral.corine' from '/home/firewise/multispectral/corine.py'>

In [2]:
(tr_list,val_list) = tools.getsets(p=0.2)
print(len(tr_list), len(val_list))
print(tr_list)
print(val_list)

20 5
[PosixPath('/home/usgs/landsat5/LT05_L1TP_195020_20060415_20161122_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_168035_20061128_20180309_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_197032_20060515_20180310_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_190017_20060717_20161120_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_199020_20060716_20161120_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_176034_20060917_20161118_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_192017_20060426_20161122_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_185033_20060730_20161120_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_169035_20060916_20161119_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_203021_20060423_20180309_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_201016_20060916_20161118_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_168035_20061214_20180309_01_T1.tif'), PosixPath('/home/usgs/landsat5/LT05_L1TP_18203

In [5]:
epoch_size = 2048
validation_size = 200
batch_size = 8
tr_loader = windows.WindowedDataset(tr_list,corine.corine_labeler,*corine.corine_attributes(), size=epoch_size).as_loader(bs=batch_size)
val_loader = windows.WindowedDataset(val_list,corine.corine_labeler,*corine.corine_attributes(), size=validation_size).as_loader(bs=batch_size)
databunch = DataBunch(tr_loader, val_loader)

In [4]:
# A very simple fully convolutional model (no pooling)
model = nn.Sequential(
    nn.Conv2d(6,20,kernel_size=3,padding=1),
    nn.ReLU(),
    nn.Conv2d(20,11,kernel_size=3,padding=1),
    nn.ReLU()
)
fastai.torch_core.init_default(model)

Sequential(
  (0): Conv2d(6, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(20, 11, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU()
)

In [15]:
learner = Learner(databunch, model, loss_func = nn.BCEWithLogitsLoss(), path='.', metrics=metrics.dice)

In [16]:
learner.fit_one_cycle(1)

epoch,train_loss,valid_loss,dice,time


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [17]:
%debug

> /home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/numpy/lib/function_base.py(4694)append()
   4692         values = ravel(values)
   4693         axis = arr.ndim-1
-> 4694     return concatenate((arr, values), axis=axis)
   4695 
   4696 

> /home/firewise/multispectral/corine.py(89)corine_labeler()
     87     cloud = ((qa & bands.PIXEL_QA['cloud']) != 0)
     88     shadow = ((qa & bands.PIXEL_QA['shadow']) != 0)
---> 89     c_data = np.append( c_data, [cloud, shadow], axis=0 )
     90 
     91     ls_data = np.delete(ls_data, bi('landsat','qa'), axis=0)  # we're done with the qa band

(9, 256, 255)
(256, 256)
(256, 256)


In [18]:
corine.tile_counts

{'/home/usgs/landsat5/LT05_L1TP_195020_20060415_20161122_01_T1.tif': {'lsempty': 311,
  'mapoverlap': 0,
  'cempty': 588,
  'dataoverlap': 10,
  'good': 51},
 '/home/usgs/landsat5/LT05_L1TP_168035_20061128_20180309_01_T1.tif': {'lsempty': 172,
  'mapoverlap': 0,
  'cempty': 0,
  'dataoverlap': 638,
  'good': 0},
 '/home/usgs/landsat5/LT05_L1TP_197032_20060515_20180310_01_T1.tif': {'lsempty': 219,
  'mapoverlap': 218,
  'cempty': 36,
  'dataoverlap': 348,
  'good': 47},
 '/home/usgs/landsat5/LT05_L1TP_190017_20060717_20161120_01_T1.tif': {'lsempty': 277,
  'mapoverlap': 21,
  'cempty': 626,
  'dataoverlap': 0,
  'good': 4},
 '/home/usgs/landsat5/LT05_L1TP_199020_20060716_20161120_01_T1.tif': {'lsempty': 310,
  'mapoverlap': 0,
  'cempty': 56,
  'dataoverlap': 561,
  'good': 33},
 '/home/usgs/landsat5/LT05_L1TP_176034_20060917_20161118_01_T1.tif': {'lsempty': 191,
  'mapoverlap': 0,
  'cempty': 646,
  'dataoverlap': 0,
  'good': 0},
 '/home/usgs/landsat5/LT05_L1TP_192017_20060426_2016112

In [42]:
ldsat = rasterio.open('/home/usgs/landsat5/LT05_L1TP_170036_20060603_20180310_01_T1.tif')